# Pull in dataset, cleaning it, and splitting it.

In [4]:
import pandas as pd 
import sklearn
from sklearn.model_selection import StratifiedShuffleSplit



#initialize to use for later
wine_train_set=0
wine_test_set=0
#code is the same as from over on the initial exploration notebook.
dataframe = pd.read_csv("winequality-white.csv")
dataframeCopy = dataframe.copy()
dataframeCopy.dropna(axis='index', how='any', inplace=True)
dataframeCopy.dropna(axis='columns', how='any', inplace=True)

#stratifying data on quality to predict it
#reverted the test size
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.25, random_state=456)

for train_indices, test_indices in splitter.split(dataframeCopy, dataframeCopy["quality"]):
    wine_train_set = dataframeCopy.iloc[train_indices]
    wine_test_set = dataframeCopy.iloc[test_indices]
    

workingset=wine_train_set.copy()
testingset=wine_test_set.copy()

# Clustering

In [13]:
#import for Clustering as pert the K-means clustering Slides.
from sklearn.cluster import KMeans
import math


#buuld classifier and X before fitting
kmeans_classifier = KMeans(n_clusters=7, n_init=20)

X= workingset[["residual sugar", "alcohol", "density","chlorides","volatile acidity", "fixed acidity", "sulphates", "pH"]]


#fit and printout attribrutes.
kmeans_classifier.fit(X)
print("Centroids are:", kmeans_classifier.cluster_centers_)
print("The final distortion is:", kmeans_classifier.inertia_)
print(kmeans_classifier.labels_)
RMS_average = math.sqrt(kmeans_classifier.inertia_ / len(X))
print("Averaging and taking the square root gives ", RMS_average)

#print out closest cluster centroid.
new_instance = workingset[["residual sugar", "alcohol", "density","chlorides","volatile acidity", "fixed acidity", "sulphates", "pH" ]]
class_out = kmeans_classifier.predict(new_instance)
print("The predicted class:", class_out)



Centroids are: [[1.30032258e+01 9.56736243e+00 9.97497495e-01 4.93282732e-02
  2.80407970e-01 7.00132827e+00 4.99829222e-01 3.13762808e+00]
 [2.20665635e+00 1.21550052e+01 9.90346494e-01 3.60433437e-02
  2.83761610e-01 6.51455108e+00 4.97260062e-01 3.21518576e+00]
 [5.65348331e+00 1.07843880e+01 9.93448331e-01 4.55152395e-02
  2.86335269e-01 6.72467344e+00 4.81320755e-01 3.20898403e+00]
 [8.90639445e+00 1.01596251e+01 9.95380755e-01 4.68582435e-02
  2.92280431e-01 6.95069337e+00 4.70847458e-01 3.16855162e+00]
 [1.72881743e+01 9.34266943e+00 9.99167884e-01 4.99253112e-02
  2.67074689e-01 6.91161826e+00 4.96265560e-01 3.14863071e+00]
 [1.58614130e+00 1.02546558e+01 9.92801087e-01 4.92793478e-02
  2.61016304e-01 7.00135870e+00 4.96315217e-01 3.21538043e+00]
 [6.58000000e+01 1.17000000e+01 1.03898000e+00 7.40000000e-02
  9.65000000e-01 7.80000000e+00 6.90000000e-01 3.39000000e+00]]
The final distortion is: 9350.752692920545
[2 1 1 ... 3 3 3]
Averaging and taking the square root gives  1.59

# Dimensional Analysis

# Vizualization and Narrative of Discoveries

# Anomalous Data

# Advanced Regression/Classifer Tools